In [ ]:
import serial
import time

class ArduinoReader:
    def __init__(self, port: str, baudrate: int = 115200, timeout: float = 1.0):
        """
        port: e.g. 'COM11' on Windows, '/dev/ttyACM0' on Linux
        baudrate: match Arduino Serial.begin()
        timeout: read timeout in seconds
        """
        self.ser = serial.Serial(port, baudrate=baudrate, timeout=timeout)
        # give Arduino time to reset when serial opens
        self.data = {'rpm': [], 'sound': [], 'lift': []}
        time.sleep(2)

    def read_line(self) -> str:
        """Read one line from Arduino, decode to string."""
        return list(map(float, self.ser.readline().decode(errors="ignore").strip().split()))

    def read_normal(self, n: int = 10) -> dict:
        """Read n normal entries (Arduino streaming at ~4 Hz)."""
        while len(self.data['rpm']) < n:
            line = self.read_line()
            if len(line) == 3:
                self.data['rpm'].append(line[0])
                self.data['sound'].append(line[1])
                self.data['lift'].append(line[2])
        return self.data
    
    def request_raw_sound(self) -> list:
        """
        Send 's' to Arduino to trigger burst mode.
        Collects until no more burst data arrives.
        """
        self.ser.write(b's')
        self.ser.flush()

        burst_data = []
        last_time = time.time()

        while True:
            line = self.read_line()
            if line:
                burst_data.append(line)
                last_time = time.time()
            else:
                # stop when nothing new for 0.2s
                if time.time() - last_time > 0.2:
                    break
        return burst_data[-16000:]

    def tare_scale(self):
        """Send a space character to Arduino (changes behaviour)."""
        self.ser.write(b' ')
        self.ser.flush()

    def close(self):
        """Close serial connection."""
        self.ser.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()


In [18]:
with ArduinoReader(port="COM11", baudrate=115200) as ar:

    print("Reading a few normal entries:")
    print(ar.read_normal(5))

    # print("Requesting burst...")
    # burst = ar.request_raw_sound()
    # print(f"Got {len(burst)} entries in burst")

    # print("Sending space...")
    # ar.tare_scale()


Reading a few normal entries:
1 [0.09]
3 [0.0, -48.25, -0.01]
3 [0.0, -45.37, 0.03]
3 [0.0, -45.84, 0.04]
3 [0.0, -45.87, -0.01]
3 [0.0, -45.85, 0.01]
{'rpm': [0.0, 0.0, 0.0, 0.0, 0.0], 'sound': [-48.25, -45.37, -45.84, -45.87, -45.85], 'lift': [-0.01, 0.03, 0.04, -0.01, 0.01]}
